In [1]:
from fsl_mrs.utils import synthetic as syn
import numpy as np
from fsl_mrs.core import MRS
import matplotlib.pyplot as plt
from fsl_mrs.utils.mrs_io import lcm_io as lcm

#dictionary with average metab concs, should be in same folder
dicty = np.load('avg_metab_concs.npy',allow_pickle='TRUE').item()

#basis file, edit accordingly
basis_path = '/path/bndu_data/fsl_mrs_simulations/basis_files/basis_file.BASIS'


In [2]:
#define different GABA conditions
gaba_base = dicty['GABA']
gaba_plus= gaba_base+0.54 #measured difference in main analysis
gaba_min = gaba_base-0.54


In [6]:
#set GABA and double check vlaue
condition = 'gaba_baseline'
snr_condition = 'snr_base'
dicty['GABA']=gaba_base
save_path = '/path/fsl_mrs_simulations/' + snr_condition +'/' + condition # edit accordingly

In [7]:
# run simulations
for i in range(2000):
    #simulate spectrum
    fid, header, _ = syn.syntheticFromBasisFile(str(basis_path),
                                                ind_scaling=['mm'],
                                                baseline=[0.0, 0.0],
                                                broadening=(28.8, 0.0),
                                                concentrations=dicty,
                                                noisecovariance=[[13.5]],
                                                bandwidth=6000.0,
                                                points=2048)

    lcm.saveRAW(save_path + '/sim_'+ condition + str(i+1) + '.RAW',fid,conj=True)

  

In [ ]:
# for estimating and matching simulation SNR
from fsl_mrs.utils.misc import parse_metab_groups, FIDToSpec, calculateAxes
from fsl_mrs.utils.fitting import fit_FSLModel
from fsl_mrs.utils import plotting
from fsl_mrs.utils import mrs_io
from fsl_mrs.utils import preproc as proc

snr_est = 53
basis,names,bheader = mrs_io.read_basis(basis_path)

snrs=np.zeros((100,1))

for i in range(100):
    #simulate spectrum
    fid, header, _ = syn.syntheticFromBasisFile(str(basis_path),
                                                ind_scaling=['mm'],
                                                baseline=[0.0, 0.0],
                                                broadening=(28.8, 0.0),
                                                concentrations=dicty,
                                                noisecovariance=[[snr_est]] ,
                                                bandwidth=6000.0,
                                                points=2048)
    # create MRS object for fitting
    mrs = MRS(FID=fid,header=header,basis=basis,names=names,basis_hdr=bheader[0])
    mrs.processForFitting(ind_scaling=['mm'])
    
    metab_groups = parse_metab_groups(mrs,'mm')
    FitArgs = {
        'model':'voigt',
        'metab_groups' : metab_groups,
        'ppmlim' : (.2,4.2),
        'baseline_order' : 4,}

    res = fit_FSLModel(mrs,**FitArgs)
    x=res.getQCParams()
    snrs[i]=x[0][13]  

/opt/anaconda3/envs/fsl_mrs/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/opt/anaconda3/envs/fsl_mrs/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning:

invalid value encountered in true_divide

/opt/anaconda3/envs/fsl_mrs/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

